Import necessary modules

In [76]:
# somehow in this file trees dont work

## for data
import pandas as pd
import random
import numpy as np

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# for ML prep amd ML
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import linear_model

Reading Data and sampling it to digestible format

In [ ]:
df_train = pd.read_csv('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/04_Term 2/10_NLP with Python/02_Assingment 4&5/train_a4_5.csv')

In [ ]:
df_test= pd.read_csv('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/04_Term 2/10_NLP with Python/02_Assingment 4&5/test_a4_5.csv')

In [ ]:
# take the sample random rather then just the top 10000
df_train_r = df_train.sample(n=10000, random_state=1)



Checking for and removing NaN in keywords

In [ ]:
# Checking how many NaN there are, then dropping
df_train_r.info()
df_train_r = df_train_r.dropna(subset=['keywords'])

In [ ]:
#Verifying if NaN are reduced
df_train_r.info()
# Sex and age given, question what to do with the ngrams
# -> tokenization has already happened, lemmatization could still be done, stopwords?

Rename DF and features for comprehensiveness

In [ ]:
df_train_r['sex'].replace('M', 1, inplace = True)
df_train_r['sex'].replace('F', 0, inplace = True)

In [ ]:
dtf = df_train_r.rename(columns = {'keywords': 'text', 'sex': 'y'})

In [ ]:
dtf.info()

See distribution for features: 'Sex'

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Number F / M", fontsize=12)
dtf["y"].reset_index().groupby("y").count().sort_values(by=
       "index").plot(kind="barh", legend=False,
        ax=ax).grid(axis='x')
plt.show()

# Difference between occurance seems fair

Bag of Words appraoch:
- reasonable as already word frequency shown in dataset

NLP - create feature matrix

In [ ]:
# creating the feature matrix
matrix = CountVectorizer(max_features=2000)
X = matrix.fit_transform(dtf.text).toarray()

In [ ]:
y = dtf.y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Sex Predicition (1 =  M, 0 = F)

Naive Bayes

In [ ]:
# Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Predict Class
y_pred_gnb = gnb.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_gnb)

# Confusion Matrix
plot_confusion_matrix(gnb, X_test, y_test)
plt.show()

# Classification report
print(classification_report(y_test, y_pred_gnb))


In [ ]:
#Logistic Regression
lr = LogisticRegression(random_state = 42, solver='liblinear')

lr.fit(X_train, y_train)

# Predict Class
y_pred_lr = lr.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_lr)

# Confusion Matrix
plot_confusion_matrix(lr, X_test, y_test)
plt.show()

# Classification report
print(classification_report(y_test, y_pred_lr))

# Prediction for hand in


Rename DF and features for comprehensiveness

In [ ]:
df_test['sex'].replace('M', 1, inplace = True)
df_test['sex'].replace('F', 0, inplace = True)

In [ ]:
dtf = df_test.rename(columns = {'keywords': 'text', 'sex': 'y'})

In [ ]:
dtf = dtf.drop(columns = ['age', 'y'])


In [ ]:
dtf = dtf.replace(np.nan, '', regex=True)
dtf.info()

In [ ]:
dtf_top_10000 = dtf.head(10000)

Bag of Words appraoch:
- reasonable as already word frequency shown in dataset

NLP - create feature matrix

In [ ]:
# creating the feature matrix
matrix = CountVectorizer(max_features=2000)
X = matrix.fit_transform(dtf_top_10000.text).toarray()

# Predict Class
Sex_pred_lr = lr.predict(X)

In [ ]:
handin = dtf_top_10000[['ID']]
handin['sex'] = Sex_pred_lr
handin

In [ ]:
age = pd.read_csv('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/04_Term 2/10_NLP with Python/02_Assingment 4&5/age.csv', index_col = False)
age = age.drop(columns = ['Unnamed: 0'])

In [ ]:
final_handin = pd.merge(age, handin, how = 'left', on = ['ID'])

In [ ]:
handin.to_csv('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/04_Term 2/10_NLP with Python/02_Assingment 4&5/final_handin.csv')
